In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import scipy.sparse as sp
import joblib
import warnings
warnings.filterwarnings('ignore')

# ── Steam-themed plot style ──
STEAM_DARK  = '#0a0e1a'
STEAM_MID   = '#1b2838'
STEAM_PANEL = '#2a475e'
STEAM_BLUE  = '#66c0f4'
STEAM_BRIGHT= '#89d4ff'
STEAM_LIGHT = '#c7d5e0'
STEAM_GREEN = '#4ade80'
STEAM_AMBER = '#fbbf24'

plt.rcParams.update({
    'figure.facecolor': STEAM_DARK, 'axes.facecolor':   STEAM_PANEL,
    'axes.edgecolor':   STEAM_BLUE, 'axes.labelcolor':  STEAM_LIGHT,
    'xtick.color':      STEAM_LIGHT,'ytick.color':      STEAM_LIGHT,
    'text.color':       STEAM_LIGHT,'grid.color':       '#3d5a73',
    'font.family':      'DejaVu Sans','axes.titlesize':  14,
    'axes.titleweight': 'bold',     'figure.dpi':       120,
})
print(' Libraries loaded.')

STEAM_BLUE  = '#66c0f4'
STEAM_BRIGHT= '#89d4ff'
STEAM_LIGHT = '#c7d5e0'
STEAM_GREEN = '#4ade80'
STEAM_AMBER = '#fbbf24'

plt.rcParams.update({
    'figure.facecolor': STEAM_DARK, 'axes.facecolor':   STEAM_PANEL,
    'axes.edgecolor':   STEAM_BLUE, 'axes.labelcolor':  STEAM_LIGHT,
    'xtick.color':      STEAM_LIGHT,'ytick.color':      STEAM_LIGHT,
    'text.color':       STEAM_LIGHT,'grid.color':       '#3d5a73',
    'font.family':      'DejaVu Sans','axes.titlesize':  14,
    'axes.titleweight': 'bold',     'figure.dpi':       120,
})
print(' Libraries loaded.')


 Libraries loaded.
 Libraries loaded.


In [2]:
# Load ENTIRE dataset — no .head() limit
games_raw = pd.read_csv('games.csv')
print(f'Full dataset shape : {games_raw.shape}')
print(f'Columns            : {list(games_raw.columns)}')
print(f'Memory usage       : {games_raw.memory_usage(deep=True).sum() / 1e6:.1f} MB')
games_raw.head(3)


Full dataset shape : (50872, 13)
Columns            : ['app_id', 'title', 'date_release', 'win', 'mac', 'linux', 'rating', 'positive_ratio', 'user_reviews', 'price_final', 'price_original', 'discount', 'steam_deck']
Memory usage       : 12.4 MB


,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck
0,13500,Prince of Persia: Warrior Within™,2008-11-21,True,False,False,Very Positive,84,2199,9.99,9.99,0.0,True
1,22364,BRINK: Agents of Change,2011-08-03,True,False,False,Positive,85,21,2.99,2.99,0.0,True
2,113020,Monaco: What's Yours Is Mine,2013-04-24,True,True,True,Very Positive,92,3722,14.99,14.99,0.0,True


In [3]:
# ── Missing Value Analysis ──
missing     = games_raw.isnull().sum()
missing_pct = (missing / len(games_raw) * 100).round(2)
missing_df  = pd.DataFrame({'Count': missing, '%': missing_pct})
missing_df  = missing_df[missing_df['Count'] > 0].sort_values('%', ascending=False)

print('=== Missing Value Report ===')
print(missing_df if not missing_df.empty else 'No missing values.')

if not missing_df.empty:
    fig, ax = plt.subplots(figsize=(10, 4))
    bars = ax.barh(missing_df.index, missing_df['%'], color=STEAM_BLUE, edgecolor=STEAM_DARK)
    ax.set_xlabel('Missing (%)')
    ax.set_title('Figure 1: Missing Value Analysis per Column')
    for bar, val in zip(bars, missing_df['%']):
        ax.text(bar.get_width()+0.1, bar.get_y()+bar.get_height()/2, f'{val}%', va='center', fontsize=9)
    plt.tight_layout()
    plt.savefig('fig01_missing_values.png', bbox_inches='tight')
    plt.show()


=== Missing Value Report ===
No missing values.


In [4]:
games = games_raw.copy()

# 1. Fill missing values
games['positive_ratio'] = games['positive_ratio'].fillna(0)
games['user_reviews']   = games['user_reviews'].fillna(0)
games['price_final']    = games['price_final'].fillna(0)
games['rating']         = games['rating'].fillna('Unknown')

# 2. Standardise column name
if 'AppID' in games.columns and 'app_id' not in games.columns:
    games.rename(columns={'AppID': 'app_id'}, inplace=True)

# 3. Outlier removal — IQR x3 on price
Q1, Q3 = games['price_final'].quantile(0.25), games['price_final'].quantile(0.75)
IQR = Q3 - Q1
before = len(games)
games = games[games['price_final'] <= Q3 + 3 * IQR].copy()
print(f'Rows removed (price outliers): {before - len(games)}')

# 4. Min-Max normalisation
scaler = MinMaxScaler()
games[['positive_ratio_norm','user_reviews_norm','price_norm']] = scaler.fit_transform(
    games[['positive_ratio','user_reviews','price_final']]
)

# 5. Ordinal rating encoding
rating_map = {
    'Overwhelmingly Positive':6,'Very Positive':5,'Mostly Positive':4,
    'Mixed':3,'Mostly Negative':2,'Very Negative':1,
    'Overwhelmingly Negative':0,'Unknown':2
}
games['rating_score'] = games['rating'].map(rating_map).fillna(3)

# 6. Platform columns
plat_cols = [c for c in ['win','mac','linux'] if c in games.columns]

# 7. Price tier
games['price_tier'] = pd.cut(
    games['price_final'],
    bins=[-0.01,0,5,15,30,60,10000],
    labels=['free','budget','mid','standard','premium','luxury']
).astype(str)

games = games.reset_index(drop=True)
print(f'\n Cleaned dataset: {games.shape}')
print(f'Memory: {games.memory_usage(deep=True).sum()/1e6:.1f} MB')
print(games['rating'].value_counts())


Rows removed (price outliers): 804

 Cleaned dataset: (50068, 18)
Memory: 16.5 MB
rating
Positive                   13393
Very Positive              12911
Mixed                      11911
Mostly Positive             8574
Mostly Negative             1817
Overwhelmingly Positive     1092
Negative                     297
Very Negative                 59
Overwhelmingly Negative       14
Name: count, dtype: int64
